### DSPT6 - Unit 3 Module 2 - Consuming Data from an API

The purpose of this notebook is to demonstrate:
- Connect to the Twitter API (and twitter_scraper) to query for tweets and user information by various parameters
- Convert tweet text using SpaCy into numerical embeddings that can be use in a predictive model

In [1]:
import tweepy

In [2]:
## ADD YOUR CREDENTIALS HERE
TWITTER_CONSUMER_API_KEY = 'u21oPsosPqUKamwd0gwBO6zf1'
TWITTER_CONSUMER_API_SECRET = 'Oa8WNzHfDlfFl2gUbEzaJY3qicN3a3O5P3YYvBzzCH4BLRFWuf'
TWITTER_ACCESS_TOKEN = '1259857629200908288-Qg2LxIvvw5W9O5eRPMQhBAE80Gutc5'
TWITTER_ACCESS_TOKEN_SECRET = 'GJHPQht7k54NBgzsP1mGZAz5GLsER9EghXSe2VunggcYb'

In [3]:
TWITTER_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY, TWITTER_CONSUMER_API_SECRET)
TWITTER_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
TWITTER = tweepy.API(TWITTER_AUTH)

In [5]:
dir(TWITTER)

[&#39;__class__&#39;,
 &#39;__delattr__&#39;,
 &#39;__dict__&#39;,
 &#39;__dir__&#39;,
 &#39;__doc__&#39;,
 &#39;__eq__&#39;,
 &#39;__format__&#39;,
 &#39;__ge__&#39;,
 &#39;__getattribute__&#39;,
 &#39;__gt__&#39;,
 &#39;__hash__&#39;,
 &#39;__init__&#39;,
 &#39;__init_subclass__&#39;,
 &#39;__le__&#39;,
 &#39;__lt__&#39;,
 &#39;__module__&#39;,
 &#39;__ne__&#39;,
 &#39;__new__&#39;,
 &#39;__reduce__&#39;,
 &#39;__reduce_ex__&#39;,
 &#39;__repr__&#39;,
 &#39;__setattr__&#39;,
 &#39;__sizeof__&#39;,
 &#39;__str__&#39;,
 &#39;__subclasshook__&#39;,
 &#39;__weakref__&#39;,
 &#39;_add_list_members&#39;,
 &#39;_lookup_friendships&#39;,
 &#39;_pack_image&#39;,
 &#39;_remove_list_members&#39;,
 &#39;_send_direct_message&#39;,
 &#39;add_list_member&#39;,
 &#39;add_list_members&#39;,
 &#39;api_root&#39;,
 &#39;auth&#39;,
 &#39;blocks&#39;,
 &#39;blocks_ids&#39;,
 &#39;cache&#39;,
 &#39;compression&#39;,
 &#39;configuration&#39;,
 &#39;create_block&#39;,
 &#39;create_favorite&#39;,
 &#39;create

In [6]:
twitter_user = TWITTER.get_user('elonmusk')

In [8]:
twitter_user.followers_count

39058219

### SpaCy Embeddings

In [4]:
import spacy

### Bringing it all together